Azimuth LUT -> https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCase/BLOCK-T74

***

Using configurable block

In [1]:
from lsst.ts.observing import ObservingBlock, ObservingScript 

In [2]:
name = "BLOCK-T74"
program = "BLOCK-T74"
reason = "SITCOM-1490"
constraints = []
scripts = []

Here we define the configuration_schema we will use to setup the configurable parts of the block. Note that it is a string

This is a conveninence function that chatGPT helped me build for the configuration schema

In [3]:
def build_configuration_schema(block_number, properties):
    """
    Builds a configuration schema string for a given BLOCK and configurable properties.

    :param block_number: The BLOCK number to include in the title and description.
    :param properties: A dictionary where each key is a property name, and each value is a dictionary
                       with keys 'description', 'type', and optionally 'default'.
    :return: A formatted configuration schema string.
    """
    # Define the base schema with the BLOCK number
    configuration_schema = (
        '$schema: http://json-schema.org/draft-07/schema#\n'
        f'title: BLOCK-{block_number} configuration\n'
        f'description: Configuration for BLOCK-{block_number}.\n'
        'type: object\n'
        'properties:\n'
    )

    # Add each property to the schema
    for prop_name, prop_details in properties.items():
        configuration_schema += f'  {prop_name}:\n'
        configuration_schema += f'    description: {prop_details["description"]}\n'
        configuration_schema += f'    type: {prop_details["type"]}\n'
        if "default" in prop_details:
            # Add quotes around the default value if it's a string
            default_value = prop_details["default"]
            if prop_details["type"] == "string":
                default_value = f'"{default_value}"'
            configuration_schema += f'    default: {default_value}\n'

    return configuration_schema

In [4]:
# Define the configurable properties that we will use in the configuration schema
properties = {
    "run_filter": {
        "description": "ComCam filter to use.",
        "type": "string",
        "default": "r_03"
    },
    "run_elevation": {
        "description": "Elevation to use for the azimuth sweep.",
        "type": "number",
        "default": 80
    }
}

# Build the configuration schema for BLOCK-404
block_number = 'T74'
configuration_schema = build_configuration_schema(block_number, properties)

# Print the resulting configuration schema
print(configuration_schema)

$schema: http://json-schema.org/draft-07/schema#
title: BLOCK-T74 configuration
description: Configuration for BLOCK-T74.
type: object
properties:
  run_filter:
    description: ComCam filter to use.
    type: string
    default: "r_03"
  run_elevation:
    description: Elevation to use for the azimuth sweep.
    type: number
    default: 80



In [5]:
azimuth_positions = [-180, -150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150, 180]

In [6]:
comcam_triplets_script = ObservingScript(
    name="maintel/take_triplet_comcam.py",
    standard=True,
    parameters= dict(
        filter="$run_filter",
        program=program,
        reason=reason
    )
)

In [7]:
stop_tracking_script = ObservingScript(
    name="maintel/stop_tracking.py",
    standard=True,
    parameters = dict()
)

In [8]:
for azimuth in azimuth_positions:
    track_target_script = ObservingScript(
        name="maintel/track_target.py",
        standard=True,
        parameters = dict(
            track_azel = dict(
                az = azimuth,
                el = "$run_elevation"
            )
        )
    )

    scripts.append(track_target_script)
    scripts.append(comcam_triplets_script)
scripts.append(stop_tracking_script)

In [9]:
block = ObservingBlock(
    name = name,
    program = program,
    configuration_schema=configuration_schema,
    scripts = scripts,
)

In [10]:
block.model_dump_json(indent=2)

output_file_path = 'BLOCKS/'+name+'.json'

with open(output_file_path, 'w') as file:
    file.write(block.model_dump_json(indent=2))